In [1]:
# Import libraries
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import warnings
import evaluate
warnings.filterwarnings("ignore", category=Warning)

In [2]:
# Load Data
data = pd.read_csv("processed_data.csv")
data.head()

,User ID,Gender,Age_Scaled,Salary_Scaled,Purchased
0,15624510,1,-1.781797,-1.490046,0
1,15810944,1,-0.253587,-1.460681,0
2,15668575,0,-1.113206,-0.785290,0
3,15603246,0,-1.017692,-0.374182,0
4,15804002,1,-1.781797,0.183751,0


In [3]:
# Define features
features = data.iloc[:, 1:-1]
X = features.to_numpy()

In [4]:
# Define target
target = data["Purchased"]
y = target.to_numpy()

In [5]:
# Transform target and features to tensors
X = torch.tensor(X)
y = torch.tensor(y)

In [6]:
# For newer versions of Python/Jupyter, change default so that method config attributes always show
from sklearn import set_config
set_config(print_changed_only=False, display='diagram')

In [7]:
# Divide data into training and testing.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=20)

In [8]:
# Create model
in_features = features.shape[1]
out_features = 1

model = nn.Sequential(
    nn.Linear(in_features, 2),
    nn.Linear(2, out_features),
    nn.Sigmoid()
)

In [9]:
# Created the dataset and the dataloader
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X_train.float(), y_train.float())
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

In [13]:
# Loss function
criterion = nn.MSELoss()

In [15]:
# Create the optimizer
import torch.optim as optim
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.85)

In [19]:
# Loop over the number of epochs and the dataloader
num_of_epochs = 1000

for i in range(num_of_epochs):
    for data in dataloader:
        #Set the gradients to zero
        optimizer.zero_grad()
        # Run a forward pass
        X_train, y_train = data
        prediction = model(X_train)
        # Calculate the loss
        loss = criterion(prediction, y_train)
        # Compute the gradients
        loss.backward()
        # Update the model's parameters
        optimizer.step()
    if (i + 1) % 100 == 0:
        print(f"Finished epoch {i}, latest loss {loss}")

Finished epoch 99, latest loss 0.3489406704902649
Finished epoch 199, latest loss 0.26889660954475403
Finished epoch 299, latest loss 0.2711690664291382
Finished epoch 399, latest loss 0.2138443887233734
Finished epoch 499, latest loss 0.24317046999931335
Finished epoch 599, latest loss 0.24188798666000366
Finished epoch 699, latest loss 0.24218744039535522
Finished epoch 799, latest loss 0.31861409544944763
Finished epoch 899, latest loss 0.24282318353652954
Finished epoch 999, latest loss 0.21339139342308044


In [21]:
# Model accuracy
with torch.no_grad():
    y_pred = model(X_test.float())

accuracy = (y_pred.round() == y).float().mean()
print(f"Model accuracy is {accuracy}")

Model accuracy is 0.6424999833106995
